In [1]:
from collections import deque
import heapq
import random
import math

In [2]:
# Definición del grafo con distancias entre las ciudades
grafo = {
    "International Falls": {"Grand Forks": 150, "Bemidji": 110, "Duluth": 200},
    "Grand Forks": {"International Falls": 150, "Fargo": 80, "Bemidji": 120},
    "Bemidji": {"International Falls": 110, "Grand Forks": 120, "St. Cloud": 140, "Duluth": 150},
    "Duluth": {"Bemidji": 150, "Minneapolis": 155, "International Falls": 200},
    "Fargo": {"Grand Forks": 80, "St. Cloud": 170, "Sioux Falls": 240},
    "St. Cloud": {"Bemidji": 140, "Fargo": 170, "Minneapolis": 65},
    "Minneapolis": {"St. Cloud": 65, "Duluth": 155, "LaCrosse": 150, "Rochester": 85, "Wausau": 180},
    "Wausau": {"Minneapolis": 180, "Green Bay": 100},
    "Green Bay": {"Wausau": 100, "Milwaukee": 120, "LaCrosse": 160},
    "LaCrosse": {"Minneapolis": 150, "Dubuque": 130, "Madison": 110, "Rochester": 70, "Green Bay": 160},
    "Rochester": {"Minneapolis": 85, "Dubuque": 130, "LaCrosse": 70, "Sioux Falls": 300},
    "Dubuque": {"Rochester": 130, "LaCrosse": 130, "Rockford": 80},
    "Rockford": {"Dubuque": 80, "Chicago": 90, "Madison": 70},
    "Chicago": {"Rockford": 90, "Milwaukee": 90},
    "Milwaukee": {"Chicago": 90, "Green Bay": 120, "Madison": 80},
    "Madison": {"Milwaukee": 80, "LaCrosse": 110, "Rockford": 70},
    "Sioux Falls": {"Fargo": 240, "Rochester": 300}
}

In [3]:
coordenadas = {
    "International Falls": (0, 10),
    "Grand Forks": (2, 8),
    "Bemidji": (4, 10),
    "Duluth": (7, 9),
    "Fargo": (3, 6),
    "St. Cloud": (6, 6),
    "Minneapolis": (8, 6),
    "Wausau": (10, 8),
    "Green Bay": (12, 8),
    "LaCrosse": (9, 5),
    "Rochester": (8, 4),
    "Dubuque": (8, 2),
    "Rockford": (9, 1),
    "Chicago": (10, 0),
    "Milwaukee": (11, 2),
    "Madison": (10, 4),
    "Sioux Falls": (1, 4)
}

In [4]:
def heuristica(ciudad_actual, ciudad_objetivo):
    x1, y1 = coordenadas[ciudad_actual]
    x2, y2 = coordenadas[ciudad_objetivo]
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

In [5]:
def bfs(grafo, inicio, objetivo):
    visitados = set()
    cola = deque([[inicio]])
    
    while cola:
        camino = cola.popleft()
        nodo = camino[-1]
        
        if nodo in visitados:
            continue
        
        if nodo == objetivo:
            return camino
        
        visitados.add(nodo)
        
        for vecino in grafo.get(nodo, {}):
            nuevo_camino = list(camino)
            nuevo_camino.append(vecino)
            cola.append(nuevo_camino)
    
    return None

In [6]:
def dfs(grafo, inicio, objetivo, camino=None, visitados=None):
    if camino is None:
        camino = [inicio]
    if visitados is None:
        visitados = set()
    
    visitados.add(inicio)
    
    if inicio == objetivo:
        return camino
    
    for vecino in grafo.get(inicio, {}):
        if vecino not in visitados:
            resultado = dfs(grafo, vecino, objetivo, camino + [vecino], visitados)
            if resultado is not None:
                return resultado
    
    return None

In [7]:
def busqueda_avara(grafo, inicio, objetivo):
    open_list = []
    heapq.heappush(open_list, (heuristica(inicio, objetivo), inicio))
    came_from = {inicio: None}
    visited = set()

    while open_list:
        _, ciudad_actual = heapq.heappop(open_list)

        if ciudad_actual == objetivo:
            path = []
            while ciudad_actual is not None:
                path.append(ciudad_actual)
                ciudad_actual = came_from[ciudad_actual]
            return path[::-1]

        visited.add(ciudad_actual)

        for vecino in grafo[ciudad_actual]:
            if vecino not in visited:
                came_from[vecino] = ciudad_actual
                heapq.heappush(open_list, (heuristica(vecino, objetivo), vecino))

    return None

In [8]:
def a_estrella(grafo, inicio, objetivo):
    open_list = []
    heapq.heappush(open_list, (0, inicio))
    came_from = {}
    g_score = {ciudad: float('inf') for ciudad in grafo}
    g_score[inicio] = 0
    f_score = {ciudad: float('inf') for ciudad in grafo}
    f_score[inicio] = heuristica(inicio, objetivo)

    while open_list:
        _, ciudad_actual = heapq.heappop(open_list)

        if ciudad_actual == objetivo:
            path = []
            while ciudad_actual in came_from:
                path.append(ciudad_actual)
                ciudad_actual = came_from[ciudad_actual]
            path.append(inicio)
            return path[::-1]

        for vecino, coste in grafo[ciudad_actual].items():
            tentative_g_score = g_score[ciudad_actual] + coste

            if tentative_g_score < g_score[vecino]:
                came_from[vecino] = ciudad_actual
                g_score[vecino] = tentative_g_score
                f_score[vecino] = g_score[vecino] + heuristica(vecino, objetivo)
                heapq.heappush(open_list, (f_score[vecino], vecino))

    return None

In [9]:
def recocido_simulado(grafo, inicio, objetivo, temperatura=10000, tasa_enfriamiento=0.003):
    ciudad_actual = inicio
    mejor_ciudad = ciudad_actual
    mejor_distancia = heuristica(ciudad_actual, objetivo)
    ruta = [ciudad_actual]

    while temperatura > 1:
        vecino = random.choice(list(grafo[ciudad_actual].keys()))
        distancia_actual = heuristica(ciudad_actual, objetivo)
        distancia_vecino = heuristica(vecino, objetivo)

        if distancia_vecino < mejor_distancia:
            mejor_ciudad = vecino
            mejor_distancia = distancia_vecino
            ruta.append(vecino)

        if distancia_vecino < distancia_actual or random.uniform(0, 1) < math.exp((distancia_actual - distancia_vecino) / temperatura):
            ciudad_actual = vecino
            if ciudad_actual not in ruta:
                ruta.append(ciudad_actual)

        temperatura *= 1 - tasa_enfriamiento

    return ruta if ruta[-1] == objetivo else None


In [10]:
def hill_climbing(grafo, inicio, objetivo):
    ciudad_actual = inicio
    mejor_ciudad = ciudad_actual
    ruta = [ciudad_actual]

    while True:
        vecinos = list(grafo[ciudad_actual].keys())
        mejor_vecino = min(vecinos, key=lambda v: heuristica(v, objetivo))
        if heuristica(mejor_vecino, objetivo) >= heuristica(ciudad_actual, objetivo):
            break
        ciudad_actual = mejor_vecino
        ruta.append(ciudad_actual)

    return ruta if ruta[-1] == objetivo else None

In [11]:
def busqueda_tabu(grafo, inicio, objetivo, tabu_tam=5):
    tabu_list = deque(maxlen=tabu_tam)
    ciudad_actual = inicio
    mejor_ciudad = ciudad_actual
    ruta = [ciudad_actual]

    while ciudad_actual != objetivo:
        vecinos = list(grafo[ciudad_actual].keys())
        mejor_vecino = None

        for vecino in vecinos:
            if vecino in tabu_list:
                continue
            if mejor_vecino is None or heuristica(vecino, objetivo) < heuristica(mejor_vecino, objetivo):
                mejor_vecino = vecino

        if mejor_vecino is None:
            break

        tabu_list.append(ciudad_actual)
        ciudad_actual = mejor_vecino
        ruta.append(ciudad_actual)

        if heuristica(ciudad_actual, objetivo) < heuristica(mejor_ciudad, objetivo):
            mejor_ciudad = ciudad_actual

    return ruta if ruta[-1] == objetivo else None


In [12]:
inicio = input("Ingresa la ciudad de inicio: ")
objetivo = input("Ingresa la ciudad objetivo: ")

if inicio not in grafo or objetivo not in grafo:
    print("Una o ambas ciudades no están en el grafo.")
else:
    print("BFS:", bfs(grafo, inicio, objetivo))
    print("DFS:", dfs(grafo, inicio, objetivo))
    print("Búsqueda Avara:", busqueda_avara(grafo, inicio, objetivo))
    print("A*:", a_estrella(grafo, inicio, objetivo))
    print("Recocido Simulado:", recocido_simulado(grafo, inicio, objetivo))
    print("Hill Climbing:", hill_climbing(grafo, inicio, objetivo))
    print("Búsqueda Tabú:", busqueda_tabu(grafo, inicio, objetivo))

BFS: ['Rochester', 'Minneapolis', 'Wausau']
DFS: ['Rochester', 'Minneapolis', 'LaCrosse', 'Dubuque', 'Rockford', 'Chicago', 'Milwaukee', 'Green Bay', 'Wausau']
Búsqueda Avara: ['Rochester', 'Minneapolis', 'Wausau']
A*: ['Rochester', 'Minneapolis', 'Wausau']
Recocido Simulado: None
Hill Climbing: ['Rochester', 'Minneapolis', 'Wausau']
Búsqueda Tabú: ['Rochester', 'Minneapolis', 'Wausau']


In [13]:
from collections import deque
import heapq
import random
import math

# Definición del grafo con distancias entre las ciudades
grafo = {
    "International Falls": {"Grand Forks": 150, "Bemidji": 110, "Duluth": 200},
    "Grand Forks": {"International Falls": 150, "Fargo": 80, "Bemidji": 120},
    "Bemidji": {"International Falls": 110, "Grand Forks": 120, "St. Cloud": 140, "Duluth": 150},
    "Duluth": {"Bemidji": 150, "Minneapolis": 155, "International Falls": 200},
    "Fargo": {"Grand Forks": 80, "St. Cloud": 170, "Sioux Falls": 240},
    "St. Cloud": {"Bemidji": 140, "Fargo": 170, "Minneapolis": 65},
    "Minneapolis": {"St. Cloud": 65, "Duluth": 155, "LaCrosse": 150, "Rochester": 85, "Wausau": 180},
    "Wausau": {"Minneapolis": 180, "Green Bay": 100},
    "Green Bay": {"Wausau": 100, "Milwaukee": 120, "LaCrosse": 160},
    "LaCrosse": {"Minneapolis": 150, "Dubuque": 130, "Madison": 110, "Rochester": 70, "Green Bay": 160},
    "Rochester": {"Minneapolis": 85, "Dubuque": 130, "LaCrosse": 70, "Sioux Falls": 300},
    "Dubuque": {"Rochester": 130, "LaCrosse": 130, "Rockford": 80},
    "Rockford": {"Dubuque": 80, "Chicago": 90, "Madison": 70},
    "Chicago": {"Rockford": 90, "Milwaukee": 90},
    "Milwaukee": {"Chicago": 90, "Green Bay": 120, "Madison": 80},
    "Madison": {"Milwaukee": 80, "LaCrosse": 110, "Rockford": 70},
    "Sioux Falls": {"Fargo": 240, "Rochester": 300}
}

coordenadas = {
    "International Falls": (0, 10),
    "Grand Forks": (2, 8),
    "Bemidji": (4, 10),
    "Duluth": (7, 9),
    "Fargo": (3, 6),
    "St. Cloud": (6, 6),
    "Minneapolis": (8, 6),
    "Wausau": (10, 8),
    "Green Bay": (12, 8),
    "LaCrosse": (9, 5),
    "Rochester": (8, 4),
    "Dubuque": (8, 2),
    "Rockford": (9, 1),
    "Chicago": (10, 0),
    "Milwaukee": (11, 2),
    "Madison": (10, 4),
    "Sioux Falls": (1, 4)
}

def heuristica(ciudad_actual, ciudad_objetivo):
    x1, y1 = coordenadas[ciudad_actual]
    x2, y2 = coordenadas[ciudad_objetivo]
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def bfs(grafo, inicio, objetivo):
    print(f"Inicio BFS desde {inicio} hasta {objetivo}")
    visitados = set()
    cola = deque([[inicio]])
    
    while cola:
        camino = cola.popleft()
        nodo = camino[-1]
        
        print(f"Explorando nodo: {nodo}, Camino actual: {camino}")
        
        if nodo in visitados:
            print(f"Nodo {nodo} ya fue visitado.")
            continue
        
        if nodo == objetivo:
            print(f"Objetivo {objetivo} alcanzado.")
            return camino
        
        visitados.add(nodo)
        
        for vecino in grafo.get(nodo, {}):
            nuevo_camino = list(camino)
            nuevo_camino.append(vecino)
            print(f"Añadiendo camino: {nuevo_camino}")
            cola.append(nuevo_camino)
    
    print("No se encontró un camino.")
    return None

def dfs(grafo, inicio, objetivo, camino=None, visitados=None):
    if camino is None:
        camino = [inicio]
    if visitados is None:
        visitados = set()
    
    print(f"Explorando DFS desde {inicio}, Camino actual: {camino}")
    visitados.add(inicio)
    
    if inicio == objetivo:
        print(f"Objetivo {objetivo} alcanzado.")
        return camino
    
    for vecino in grafo.get(inicio, {}):
        if vecino not in visitados:
            print(f"Visitando vecino: {vecino}")
            resultado = dfs(grafo, vecino, objetivo, camino + [vecino], visitados)
            if resultado is not None:
                return resultado
    
    print(f"Retorno sin éxito desde {inicio}.")
    return None

def busqueda_avara(grafo, inicio, objetivo):
    print(f"Inicio Búsqueda Avara desde {inicio} hasta {objetivo}")
    open_list = []
    heapq.heappush(open_list, (heuristica(inicio, objetivo), inicio))
    came_from = {inicio: None}
    visited = set()

    while open_list:
        _, ciudad_actual = heapq.heappop(open_list)
        print(f"Explorando ciudad actual: {ciudad_actual}")

        if ciudad_actual == objetivo:
            path = []
            while ciudad_actual is not None:
                path.append(ciudad_actual)
                ciudad_actual = came_from[ciudad_actual]
            print(f"Objetivo alcanzado. Camino: {path[::-1]}")
            return path[::-1]

        visited.add(ciudad_actual)

        for vecino in grafo[ciudad_actual]:
            if vecino not in visited:
                came_from[vecino] = ciudad_actual
                print(f"Añadiendo {vecino} a la lista de exploración.")
                heapq.heappush(open_list, (heuristica(vecino, objetivo), vecino))

    print("No se encontró un camino.")
    return None

def a_estrella(grafo, inicio, objetivo):
    print(f"Inicio A* desde {inicio} hasta {objetivo}")
    open_list = []
    heapq.heappush(open_list, (0, inicio))
    came_from = {}
    g_score = {ciudad: float('inf') for ciudad in grafo}
    g_score[inicio] = 0
    f_score = {ciudad: float('inf') for ciudad in grafo}
    f_score[inicio] = heuristica(inicio, objetivo)

    while open_list:
        _, ciudad_actual = heapq.heappop(open_list)
        print(f"Explorando ciudad actual: {ciudad_actual}")

        if ciudad_actual == objetivo:
            path = []
            while ciudad_actual in came_from:
                path.append(ciudad_actual)
                ciudad_actual = came_from[ciudad_actual]
            path.append(inicio)
            print(f"Objetivo alcanzado. Camino: {path[::-1]}")
            return path[::-1]

        for vecino, coste in grafo[ciudad_actual].items():
            tentative_g_score = g_score[ciudad_actual] + coste

            if tentative_g_score < g_score[vecino]:
                came_from[vecino] = ciudad_actual
                g_score[vecino] = tentative_g_score
                f_score[vecino] = g_score[vecino] + heuristica(vecino, objetivo)
                print(f"Actualizando {vecino}, g_score: {tentative_g_score}, f_score: {f_score[vecino]}")
                heapq.heappush(open_list, (f_score[vecino], vecino))

    print("No se encontró un camino.")
    return None

def recocido_simulado(grafo, inicio, objetivo, temperatura=10000, tasa_enfriamiento=0.003):
    print(f"Inicio Recocido Simulado desde {inicio} hasta {objetivo}")
    ciudad_actual = inicio
    mejor_ciudad = ciudad_actual
    mejor_distancia = heuristica(ciudad_actual, objetivo)
    ruta = [ciudad_actual]

    while temperatura > 1:
        vecino = random.choice(list(grafo[ciudad_actual].keys()))
        distancia_actual = heuristica(ciudad_actual, objetivo)
        distancia_vecino = heuristica(vecino, objetivo)
        print(f"Temperatura: {temperatura}, Ciudad actual: {ciudad_actual}, Vecino elegido: {vecino}")

        if distancia_vecino < mejor_distancia:
            mejor_ciudad = vecino
            mejor_distancia = distancia_vecino
            ruta.append(vecino)
            print(f"Mejor distancia encontrada: {mejor_distancia} en {mejor_ciudad}")

        if distancia_vecino < distancia_actual or random.uniform(0, 1) < math.exp((distancia_actual - distancia_vecino) / temperatura):
            ciudad_actual = vecino
            if ciudad_actual not in ruta:
                ruta.append(ciudad_actual)

        temperatura *= 1 - tasa_enfriamiento

    print(f"Ruta final: {ruta}")
    return ruta if ruta[-1] == objetivo else None

def hill_climbing(grafo, inicio, objetivo):
    print(f"Inicio Hill Climbing desde {inicio} hasta {objetivo}")
    ciudad_actual = inicio
    ruta = [ciudad_actual]

    while True:
        vecinos = list(grafo[ciudad_actual].keys())
        mejor_vecino = min(vecinos, key=lambda v: heuristica(v, objetivo))
        print(f"Ciudad actual: {ciudad_actual}, Vecino seleccionado: {mejor_vecino}")

        if heuristica(mejor_vecino, objetivo) >= heuristica(ciudad_actual, objetivo):
            print("No se encontró un mejor vecino; se detiene la búsqueda.")
            break

        ciudad_actual = mejor_vecino
        ruta.append(ciudad_actual)

        if ciudad_actual == objetivo:
            print("Objetivo alcanzado.")
            return ruta

    print(f"Ruta final: {ruta}")
    return ruta if ruta[-1] == objetivo else None

def busqueda_tabu(grafo, inicio, objetivo, tabu_tam=5):
    print(f"Inicio Búsqueda Tabú desde {inicio} hasta {objetivo}")
    tabu_list = deque(maxlen=tabu_tam)
    ciudad_actual = inicio
    ruta = [ciudad_actual]

    while ciudad_actual != objetivo:
        vecinos = list(grafo[ciudad_actual].keys())
        mejor_vecino = None

        for vecino in vecinos:
            if vecino in tabu_list:
                print(f"Vecino {vecino} está en la lista Tabú y no será explorado.")
                continue
            if mejor_vecino is None or heuristica(vecino, objetivo) < heuristica(mejor_vecino, objetivo):
                mejor_vecino = vecino
                print(f"Nuevo mejor vecino encontrado: {mejor_vecino}")

        if mejor_vecino is None:
            print("No hay vecinos disponibles fuera de la lista Tabú.")
            break

        tabu_list.append(ciudad_actual)
        ciudad_actual = mejor_vecino
        ruta.append(ciudad_actual)
        print(f"Ciudad actual: {ciudad_actual}, Ruta: {ruta}")

        if ciudad_actual == objetivo:
            print("Objetivo alcanzado.")
            return ruta

    print(f"Ruta final: {ruta}")
    return ruta if ruta[-1] == objetivo else None

# Llamadas de ejemplo
inicio = input("Ingresa la ciudad de inicio: ")
objetivo = input("Ingresa la ciudad objetivo: ")

if inicio not in grafo or objetivo not in grafo:
    print("Una o ambas ciudades no están en el grafo.")
else:
    print("\nBFS:", bfs(grafo, inicio, objetivo), "\n")
    print("\nDFS:", dfs(grafo, inicio, objetivo), "\n")
    print("\nBúsqueda Avara:", busqueda_avara(grafo, inicio, objetivo), "\n")
    print("\nA*:", a_estrella(grafo, inicio, objetivo), "\n")
    print("\nRecocido Simulado:", recocido_simulado(grafo, inicio, objetivo), "\n")
    print("\nHill Climbing:", hill_climbing(grafo, inicio, objetivo), "\n")
    print("\nBúsqueda Tabú:", busqueda_tabu(grafo, inicio, objetivo), "\n")


Inicio BFS desde Rochester hasta Wausau
Explorando nodo: Rochester, Camino actual: ['Rochester']
Añadiendo camino: ['Rochester', 'Minneapolis']
Añadiendo camino: ['Rochester', 'Dubuque']
Añadiendo camino: ['Rochester', 'LaCrosse']
Añadiendo camino: ['Rochester', 'Sioux Falls']
Explorando nodo: Minneapolis, Camino actual: ['Rochester', 'Minneapolis']
Añadiendo camino: ['Rochester', 'Minneapolis', 'St. Cloud']
Añadiendo camino: ['Rochester', 'Minneapolis', 'Duluth']
Añadiendo camino: ['Rochester', 'Minneapolis', 'LaCrosse']
Añadiendo camino: ['Rochester', 'Minneapolis', 'Rochester']
Añadiendo camino: ['Rochester', 'Minneapolis', 'Wausau']
Explorando nodo: Dubuque, Camino actual: ['Rochester', 'Dubuque']
Añadiendo camino: ['Rochester', 'Dubuque', 'Rochester']
Añadiendo camino: ['Rochester', 'Dubuque', 'LaCrosse']
Añadiendo camino: ['Rochester', 'Dubuque', 'Rockford']
Explorando nodo: LaCrosse, Camino actual: ['Rochester', 'LaCrosse']
Añadiendo camino: ['Rochester', 'LaCrosse', 'Minneapoli